# Step-C 预测

### 1.加载CONV层参数，加载FC层参数

### 2.运行模型预测Submit数据

### 3.写入CSV文件

In [1]:
import os
import random
import tensorflow as tf 
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from data_preprocess import *
from image_preprocess import *
from bcnn_model import *

In [2]:
weights_file = "vgg16_weights.npz"
weights_file_last_layer = "bcnn_last_weights.npz"
weights_file_conv = "bcnn_conv_weights.npz"

image_size = 224
n_classes = 12
submit_size = 794
submit_batch_size = 32

In [3]:
test = [0,1,2,3,4,5,6,7,8]
print(test[:2])
print(test[2:])
print(test[:-2])

[0, 1]
[2, 3, 4, 5, 6, 7, 8]
[0, 1, 2, 3, 4, 5, 6]


In [4]:
tf.reset_default_graph()
sess = tf.Session()
# build input
imgs = tf.placeholder(tf.float32, shape = [None, image_size, image_size, 3], name = "input_x")
keep_prob = tf.placeholder(tf.float32, name = "keep_prob")

# build cal graph
bilinear_cnn = vgg_bilinear_model(imgs, 
                                  keep_prob, 
                                  weights_file = weights_file_conv, 
                                  weights_file_last = weights_file_last_layer, 
                                  sess = sess, 
                                  finetune = True)

# build cal node parameter
predicted = tf.nn.softmax(bilinear_cnn.fc3l, name = "predicted")

# start train
sess.run(tf.global_variables_initializer())
bilinear_cnn.load_all_own_weights(sess)

for v in tf.trainable_variables():
    print("Trainable variables", v)
print('Starting predicting')


###################################
submit_img_batch = submit_get_batchs(tfrecord_filename = "submit.tfrecords", 
                                            image_size = image_size, 
                                            batch_size = submit_batch_size)

coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord)

submit_predicted = []
img_num = 0

for i in range(int(submit_size / submit_batch_size) + 1):
    print(1)
    img_num = img_num + submit_batch_size

    # get data
    submit_imgs = sess.run(submit_img_batch)
    
    # at the end of data
    if (img_num > submit_size):
        submit_imgs = submit_imgs[ :-(img_num - submit_size)]
    
    print(2)
    # predict single loop
    predicted_result = sess.run(predicted, feed_dict={imgs: submit_imgs, keep_prob: 1.})
    print(3)
    predicted_result = list(predicted_result)
    submit_predicted.extend(predicted_result)
    
    print('completed', i)

# close data queue
coord.request_stop()
coord.join(threads)

# end
sess.close()

Adding Data Augmentation


D:\Users\XX\Anaconda2\envs\gpu-env-tf\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Shape of z (?, 262144)
conv_layer [[[[ 4.79724169e-01 -1.77854285e-01  3.73495892e-02 ... -1.19056016e-01
    -4.86118235e-02  3.06583364e-02]
   [ 5.51010847e-01  1.77014191e-02  9.87275243e-02 ... -7.65096173e-02
    -5.11239022e-02  3.40547599e-02]
   [ 4.33386236e-01  1.11716330e-01  3.39476205e-02 ... -1.23892114e-01
    -5.15401624e-02  2.80593429e-03]]

  [[ 4.08727527e-01 -1.75623596e-01 -5.17894607e-03 ... -1.14073545e-01
    -2.74549335e-01 -4.30106670e-02]
   [ 4.40814644e-01  4.41833548e-02  5.12870811e-02 ... -9.06181559e-02
    -2.96373188e-01 -7.38389939e-02]
   [ 3.77239496e-01  1.56427950e-01  1.65483530e-03 ... -1.39607966e-01
    -2.33262092e-01 -6.75706193e-02]]

  [[-6.46550357e-02 -1.59813732e-01 -1.38237387e-01 ... -1.21087477e-01
    -4.60065842e-01 -3.27527791e-01]
   [-8.09265971e-02  3.83964740e-02 -1.01794742e-01 ... -8.62325355e-02
    -5.05873263e-01 -3.87156606e-01]
   [-5.80082461e-02  1.29876092e-01 -1.15684174e-01 ... -1.31812900e-01
    -3.75346690e-0

In [6]:
pic_src = "E:/PROJECT/kaggle-seedling-classification/dataset/test/"
all_files = os.listdir(pic_src)
print(len(submit_predicted))

class_index = ['Black-grass', 'Charlock', 
               'Cleavers', 'Common Chickweed', 
               'Common wheat', 'Fat Hen', 
               'Loose Silky-bent', 'Maize', 
               'Scentless Mayweed', 'Shepherds Purse', 
               'Small-flowered Cranesbill', 'Sugar beet']

import csv  

def as_num(x):
    y = '{:.6f}'.format(x) # 6f表示保留6位小数点的float型
    return(y)

with open('output.csv', 'w+', newline='') as csv_file:  
    writer = csv.writer(csv_file)
    writer.writerow(["file", "species"])
    for index, name in enumerate(all_files):
        class_name = class_index[np.argmax(submit_predicted[index])]
        writer.writerow([name, class_name])

794
